In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
import scipy, scipy.optimize
import numpy as np
import astro_dynamo
from astro_dynamo.snap import ParticleType
%aimport  astro_dynamo.axisymmetric_potentials


Check we reproduce the same answer as galpy for some random potential

In [ ]:
import galpy.potential
q=0.5
galpy_pot = galpy.potential.TwoPowerTriaxialPotential(c=q,ro=1,vo=1)
galpy_pot.turn_physical_off()
pot = astro_dynamo.axisymmetric_potentials.SpheroidalPotential(lambda m: galpy_pot._amp*galpy_pot._mdens(m),q=q)

In [ ]:
x=np.linspace(0,10,100)
plt.semilogy(x,list(map(lambda x: -galpy_pot.Rforce(x,1),x)),'r',label='galpy FR')
plt.semilogy(x,-pot.f_r_cyl(x,np.array([1.])),'--k')
plt.semilogy(x,list(map(lambda x: -galpy_pot.zforce(x,1),x)),'y',label='galpy Fz')
plt.semilogy(x,-pot.f_z(x,np.array([1.])),'--k',label='astro-dynamo')
plt.legend()
plt.ylabel('Force')
plt.xlabel('R')

In [ ]:
x=np.linspace(0,10,100)
plt.plot(x,list(map(lambda x: galpy_pot.vcirc(x,0),x)),'r',label='galpy FR')
plt.plot(x,torch.sqrt(pot.vc2(x,np.array([0.]))),'--k',label='astro-dynamo')
plt.ylabel('Vc')
plt.xlabel('R')

Try replacing the dark matter particles in a snapshot by an analytic profile

In [ ]:
snap=astro_dynamo.snap.SnapShot('../inputmodels/M85_0.gz',omega=1.,
              particle_type_mapping={0:ParticleType.DarkMatter,1:ParticleType.Star})

In [ ]:
# Let's see what the profile acutually looks like in the model
colwidth=7
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(colwidth, colwidth*1.6))
rbins=np.arange(1,20,2)
q=[]
qerr=[]
for ir in range(len(rbins)-1):
    i=((snap.dm.r > rbins[ir]) & (snap.dm.r <= rbins[ir+1]))
    st=snap.dm.z[i]/snap.dm.r[i]
    mintheta=np.sin(30./180*np.pi)
    H,edges=np.histogram(st**2,np.linspace(mintheta**2,1,100),weights=snap.dm.masses[i])
    x=0.5*(edges[:-1]+edges[1:])
    ctedges=1-np.sqrt(edges)
    vol = ctedges[:-1]-ctedges[1:]
    rho=H/vol
    def f(x,a,b):
        return a*(b*x + 1)
    popt,pcov=scipy.optimize.curve_fit(f,x,rho,p0=[-0.8,1.0],sigma=0.1*rho)
    perr=np.sqrt(np.diag(pcov))
    q.append(1/np.sqrt(1-popt[1]))
    qerr.append(q[-1]*0.5*perr[1]/popt[1])
q=np.array(q)
qerr=np.array(qerr)

r=0.5*(rbins[:-1]+rbins[1:])
axes[0].errorbar(r,q,yerr=qerr)
axes[0].set_ylabel(r'$q_\rho$')
axes[0].set_xlabel(r'$r$ [kpc]')
i=(r>2) & (r<20)
qfix = np.mean(q[i])
axes[0].plot([2,20],[qfix,qfix])
print('q={:.3f}'.format(qfix))


m = np.sqrt((snap.dm.rcyl)**2 + (snap.dm.z/qfix)**2)
(H,medge) = np.histogram(m,np.linspace(0.1,20,50),weights=snap.dm.masses)
volcorr=np.arcsinh(np.sqrt(1/qfix**2 - 1))/np.sqrt(1/qfix**2 - 1)
volcorr=qfix
print(volcorr)
vol=4*np.pi*(medge[1:]**3 - medge[:-1]**3)/3*volcorr
mmid=0.5*(medge[1:] + medge[:-1])
axes[1].plot(mmid,H/vol)
axes[1].set_yscale('log')
def ein(m,rhor0,m0,alpha):
    rho0 = rhor0 / (np.exp(-(2 / alpha) * ((8.2 / m0) ** alpha - 1)))
    return rho0 * np.exp(-(2 / alpha) * ((m / m0) ** alpha - 1))
rho=H/vol
popt,pcov=scipy.optimize.curve_fit(ein,mmid,rho,p0=[1e7,8.0,0.7],sigma=0.01*rho)
print(popt)
print(np.sqrt(np.diag(pcov)))
axes[1].plot(mmid,ein(mmid,*popt))
axes[1].set_xlabel(r'$m [{\rm kpc}]$')
axes[1].set_ylabel(r'$\rho [M_\odot\,{\rm kpc}^{-3}]$')
truths=np.hstack((popt,qfix))

In [ ]:
pot = astro_dynamo.axisymmetric_potentials.SpheroidalPotential(lambda m: ein(m,*popt),q=qfix)
pot.f_r_cyl(torch.tensor([[10.,10.],[5.,5.]]),torch.tensor([[10.,5.],[10.,5.]]))

In [ ]:
import mwtools.nemo

In [ ]:
x=np.linspace(0)
grav = mwtools.nemo.gravity_cartesian_grid(snap.as_numpy_array(), potential.x,potential.y,[0.])